In [1]:
# !pip install torchtext
is_kaggle = False
working_dir = '/kaggle/input/aml-dataset' if is_kaggle else '../datasets'

## Todo:
- Переписать метрики
- Перебор hyperparameters

## Пути улучшения
- Провести эксперименты со всеми датасетами
- Модификация RNN, LSTM
- Взять модель с Hugging Face
- Изменить embedings (larger Glove, twiter glove, trainable glove)
- Перебор hyperparameters
    > lr

In [2]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("wanb")
secret_value_0 = "write_your_secret_wandb_key"
secret_value_0 = "5f908c81529fb55aaa472155b075ca91804dac20"

import wandb

wandb.login(key=secret_value_0)

wandb: Currently logged in as: shalagin-danil (my_own_opt). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\89123\.netrc


True

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchtext
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

from tqdm.auto import tqdm
import gc

## Text preprocessing

In [4]:
def str_to_list(value):
    list_values = value.strip('[]').split(', ')
    cleaned_list_values = [item[1:-1] for item in list_values]
    return cleaned_list_values

In [5]:
def str_context_to_list(value):
    list_values = value.strip('[]').split(' ')
    cleaned_list_values = []
    for item in list_values:
        item.strip()
        # if '\n' in item:
        #     item.replace('\n', '')
        if item != "":
            cleaned_list_values.append(float(item))
    return cleaned_list_values

In [6]:
# dataset = pd.read_csv("../datasets/tonetags_wsd_1.csv", index_col=0, converters={"text": str_to_list})
# dataset.tags.nunique()/len(dataset)
# labels = dataset1.tags.unique().tolist()

In [7]:
max_length = 2048
batch_size = 32

In [8]:
def get_glove_6b():
    embeding_object = torchtext.vocab.GloVe('6B', dim=50)
    vec_6 = embeding_object.vectors.numpy()
    vec_6 = np.append(vec_6, np.zeros(50)).reshape(-1, 50)
    vec_6 = np.append(vec_6, np.ones(50)).reshape(-1, 50)

    vocab_6 = embeding_object.stoi
    vocab_6["<unk>"] = len(vocab_6)
    vocab_6["<pad>"] = len(vocab_6)

    embed_tensor_6 = torch.tensor(vec_6, dtype=torch.float)
    embed_glove_6 = nn.Embedding.from_pretrained(embed_tensor_6, freeze=True)

    embed_size_6 = len(vocab_6)

    return embed_glove_6, vocab_6, embed_size_6

In [9]:
class myDataset(Dataset):
    def __init__(self, dataset, vocab=None):
        if vocab is None:
            _, vocab, _ = get_glove_6b()
        self.data = []

        for sentence in dataset.text:
            if len(sentence) > max_length:
                continue
            sentence_ids = []
            for token in sentence:
                try:
                    sentence_ids.append(vocab[token])
                except KeyError:
                    sentence_ids.append(vocab["<unk>"])
            self.data.append(sentence_ids)
        dataset = dataset.reset_index(drop=True)
        self.labels = dataset.tags

        self.context = None

        if 'context' in dataset.columns:
            self.context = dataset.context
        #     self.context = self.context[:1000]
        #     
        # self.data = self.data[:1000]
        # self.labels = self.labels[:1000]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.context is None:
            return self.data[idx], torch.tensor(self.labels.iloc[idx])
        else:
            return self.data[idx], torch.tensor(self.labels.iloc[idx]), self.context.iloc[idx]

In [10]:
vocab_collate_fn = None


def collate_fn(batch):
    global vocab_collate_fn
    vocab = vocab_collate_fn
    data_ids = []
    labels = []
    for dat in batch:
        data_ids.append(dat[0])
        labels.append(dat[1])

    for i in range(len(data_ids)):
        while len(data_ids[i]) < max_length:
            data_ids[i].append(vocab["<pad>"])

    return data_ids, labels


In [11]:
# vec = torchtext.vocab.GloVe('6B', dim=50).vectors.numpy()

In [12]:
def get_glove_6b():
    embeding_object = torchtext.vocab.GloVe('6B', dim=50)
    vec_6 = embeding_object.vectors.numpy()
    vec_6 = np.append(vec_6, np.zeros(50)).reshape(-1, 50)
    vec_6 = np.append(vec_6, np.ones(50)).reshape(-1, 50)

    vocab_6 = embeding_object.stoi
    vocab_6["<unk>"] = len(vocab_6)
    vocab_6["<pad>"] = len(vocab_6)

    embed_tensor_6 = torch.tensor(vec_6, dtype=torch.float)
    embed_glove_6 = nn.Embedding.from_pretrained(embed_tensor_6, freeze=True)

    embed_size_6 = len(vocab_6)

    return embed_glove_6, vocab_6, embed_size_6

In [13]:
def get_glove_42b():
    embeding_object = torchtext.vocab.GloVe('42B', dim=300)
    vec_840 = embeding_object.vectors.numpy()
    vec_840 = np.append(vec_840, np.zeros(300)).reshape(-1, 300)
    vec_840 = np.append(vec_840, np.ones(300)).reshape(-1, 300)

    vocab_840 = embeding_object.stoi
    vocab_840["<unk>"] = len(vocab_840)
    vocab_840["<pad>"] = len(vocab_840)

    embed_tensor = torch.tensor(vec_840, dtype=torch.float)
    embed_glove_840 = nn.Embedding.from_pretrained(embed_tensor, freeze=True)

    embed_size_840 = len(vec_840)

    return embed_glove_840, vocab_840, embed_size_840

In [14]:
def get_glove_twitter():
    embeding_object = torchtext.vocab.GloVe('twitter.27B', dim=50)
    vec_twitter = embeding_object.vectors.numpy()
    vec_twitter = np.append(vec_twitter, np.zeros(50)).reshape(-1, 50)
    vec_twitter = np.append(vec_twitter, np.ones(50)).reshape(-1, 50)

    vocab_twitter = embeding_object.stoi
    vocab_twitter["<unk>"] = len(vocab_twitter)
    vocab_twitter["<pad>"] = len(vocab_twitter)

    embed_tensor = torch.tensor(vec_twitter, dtype=torch.float)
    embed_glove_twitter = nn.Embedding.from_pretrained(embed_tensor, freeze=True)

    embed_size_twitter = len(vec_twitter)

    return embed_glove_twitter, vocab_twitter, embed_size_twitter

## WSD 1

In [15]:
# # dataset = pd.read_csv("../datasets/tonetags_wsd_1.csv", index_col=0, converters={"text": str_to_list})
# dataset_wsd_1 = pd.read_csv("/kaggle/input/aml-dataset/tonetags_wsd_1.csv", index_col=0, converters={"text": str_to_list})
# labels = dataset_wsd_1.tags.unique().tolist()
# dataset_wsd_1.tags = dataset_wsd_1.tags.apply(labels.index)

In [16]:
# dataset_wsd_1

In [17]:
# train, test = train_test_split(dataset_wsd_1, test_size=0.2, shuffle=True,stratify=dataset_wsd_1.tags)

# train_dataset1 = myDataset(train)
# test_dataset1 = myDataset(test)

In [18]:
def wsd1_creator(embeding):
    dataset_wsd_1 = pd.read_csv(f"{working_dir}/tonetags_wsd_1.csv", index_col=0, converters={"text": str_to_list})

    labels = dataset_wsd_1.tags.unique().tolist()
    dataset_wsd_1.tags = dataset_wsd_1.tags.apply(labels.index)

    train, test = train_test_split(dataset_wsd_1, test_size=0.2, shuffle=True, stratify=dataset_wsd_1.tags)

    train, test = myDataset(train, embeding), myDataset(test, embeding)
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)
    return train_loader, test_loader

In [19]:
# train_dataloader1 = DataLoader(train_dataset1, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
# test_dataloader1 = DataLoader(test_dataset1, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [20]:
# for batch in tqdm(train_dataloader):
#     embedded_tokens = batch[0]
#     labels = batch[1]

## WSD 2

In [21]:
# dataset_wsd_2 = pd.read_csv("/kaggle/input/aml-dataset/tonetags_wsd_2.csv", index_col=0, converters={"text": str_to_list})
# dataset_wsd_2 = dataset_wsd_2.dropna()

# labels = dataset_wsd_2.tags.unique().tolist()
# dataset_wsd_2.tags = dataset_wsd_2.tags.apply(labels.index)

# dataset_wsd_2.context = dataset_wsd_2.context.apply(str_context_to_list)

# dataset_wsd_2.head(5)

In [22]:
# train_wsd_2, test_wsd_2 = train_test_split(dataset_wsd_2, stratify=dataset_wsd_2['tags'], test_size=0.2, random_state=42)
# train_dataset_wsd_2, test_dataset_wsd_2 = myDataset(train_wsd_2), myDataset(test_wsd_2)

In [23]:
def wsd2_creator(embeding):
    dataset_wsd_2 = pd.read_csv(f"{working_dir}/tonetags_wsd_2.csv", index_col=0, converters={"text": str_to_list})
    dataset_wsd_2 = dataset_wsd_2.dropna()

    labels = dataset_wsd_2.tags.unique().tolist()
    dataset_wsd_2.tags = dataset_wsd_2.tags.apply(labels.index)
    dataset_wsd_2.context = dataset_wsd_2.context.apply(str_context_to_list)

    train_wsd_2, test_wsd_2 = train_test_split(dataset_wsd_2, stratify=dataset_wsd_2['tags'], test_size=0.2,
                                               random_state=42)

    train, test = myDataset(train_wsd_2, embeding), myDataset(test_wsd_2, embeding)
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)
    return train_loader, test_loader

In [24]:
# train_dataloader_wsd_2 = DataLoader(train_dataset_wsd_2, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)
# test_dataloader_wsd_2 = DataLoader(test_dataset_wsd_2, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)

## Clean_corrected dataset

In [25]:
# dataset_clean_corrected = pd.read_csv("/kaggle/input/aml-dataset/tonetags_dataset_tumblr_clean_corrected_text.csv", converters={"text": str_to_list})

# labels = dataset_clean_corrected.tags.unique().tolist()
# dataset_clean_corrected.tags =dataset_clean_corrected.tags.apply(labels.index)

# dataset_clean_corrected.head(5)

In [26]:
# train_clean_corrected, test_clean_corrected = train_test_split(dataset_clean_corrected, stratify=dataset_clean_corrected['tags'], test_size=0.2, random_state=42)
# train_dataset_clean_corrected, test_dataset_clean_corrected = myDataset(train_clean_corrected), myDataset(test_clean_corrected)

In [27]:
def clean_corrected_creator(embeding):
    dataset_clean_corrected = pd.read_csv(f"{working_dir}/tonetags_dataset_tumblr_clean_corrected_text.csv",
                                          converters={"text": str_to_list})

    labels = dataset_clean_corrected.tags.unique().tolist()
    dataset_clean_corrected.tags = dataset_clean_corrected.tags.apply(labels.index)

    train_clean_corrected, test_clean_corrected = train_test_split(dataset_clean_corrected,
                                                                   stratify=dataset_clean_corrected['tags'],
                                                                   test_size=0.2, random_state=42)

    train, test = myDataset(train_clean_corrected, embeding), myDataset(test_clean_corrected, embeding)
    train_dataloader_clean_corrected = DataLoader(train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn,
                                                  drop_last=True)
    test_dataloader_clean_corrected = DataLoader(test, batch_size=batch_size, shuffle=True, collate_fn=collate_fn,
                                                 drop_last=True)
    return train_dataloader_clean_corrected, test_dataloader_clean_corrected

In [28]:
# train_dataloader_clean_corrected = DataLoader(train_dataset_clean_corrected, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)
# test_dataloader_clean_corrected = DataLoader(test_dataset_clean_corrected, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)

## Clean

In [29]:
# dataset_clean = pd.read_csv("/kaggle/input/aml-dataset/tonetags_dataset_tumblr_clean.csv", converters={"text": str_to_list})

# labels = dataset_clean.tags.unique().tolist()
# dataset_clean.tags =dataset_clean.tags.apply(labels.index)

# dataset_clean.head(5)

In [30]:
# train_clean, test_clean = train_test_split(dataset_clean, stratify=dataset_clean['tags'], test_size=0.2, random_state=42)
# train_dataset_clean, test_dataset_clean = myDataset(train_clean), myDataset(test_clean)

In [31]:
def clean_creator(embeding):
    dataset_clean = pd.read_csv(f"{working_dir}/tonetags_dataset_tumblr_clean.csv", converters={"text": str_to_list})

    labels = dataset_clean.tags.unique().tolist()
    dataset_clean.tags = dataset_clean.tags.apply(labels.index)

    train_clean, test_clean = train_test_split(dataset_clean, stratify=dataset_clean['tags'], test_size=0.2,
                                               random_state=42)

    train, test = myDataset(train_clean, embeding), myDataset(test_clean, embeding)
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)
    return train_loader, test_loader

In [32]:
# train_dataloader_clean = DataLoader(train_dataset_clean, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)
# test_dataloader_clean = DataLoader(test_dataset_clean, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)

In [33]:
def clean_corrected_wsd2_creator(embeding):
    dataset_clean_corrected = pd.read_csv(f"{working_dir}/tonetags_dataset_tumblr_clean_corrected_text_wsd_2.csv",
                                          converters={"text": str_to_list})

    labels = dataset_clean_corrected.tags.unique().tolist()
    dataset_clean_corrected.tags = dataset_clean_corrected.tags.apply(labels.index)

    train_clean_corrected, test_clean_corrected = train_test_split(dataset_clean_corrected,
                                                                   stratify=dataset_clean_corrected['tags'],
                                                                   test_size=0.2, random_state=42)

    train, test = myDataset(train_clean_corrected, embeding), myDataset(test_clean_corrected, embeding)
    train_dataloader_clean_corrected = DataLoader(train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn,
                                                  drop_last=True)
    test_dataloader_clean_corrected = DataLoader(test, batch_size=batch_size, shuffle=True, collate_fn=collate_fn,
                                                 drop_last=True)
    return train_dataloader_clean_corrected, test_dataloader_clean_corrected


## Train utils

In [34]:
epoch_result = {}


def update_result(d):
    global epoch_result

    for k in d.keys():
        if k in list(epoch_result.keys()):
            raise Exception

    epoch_result.update(d)


def log():
    global epoch_result
    wandb.log(epoch_result)
    epoch_result.clear()

In [35]:
import torchmetrics
def train_epoch(trainloader, model, opt, loss_criterion):
    global epoch_result
    global device
    model.to(device)
    model.train()
    #loss для одной epoch
    train_loss = 0
    total = 0
    correct = 0

    #применить для каждого бача из trainload

    start_time = datetime.datetime.now()
    number_of_batches = len(list(trainloader))
    result = []
    result1 = []
    for batch_idx, inputs_all in enumerate(trainloader):

        inputs, targets, context = None, None, None
        if len(inputs_all) == 2:
            inputs, targets = inputs_all
            inputs = torch.tensor(inputs)
            targets = torch.tensor(targets)
            inputs, targets = inputs.to(device), targets.to(device)
        else:
            inputs, targets, context = inputs_all
            inputs = torch.tensor(inputs)
            targets = torch.tensor(targets)
            context = torch.tensor(context)
            inputs, targets, context = inputs.to(device), targets.to(device), context.to(device)
        #тренировка
        opt.zero_grad()

        outputs = model(inputs, context).reshape(len(inputs), -1)

        #шаг оптимизации/loss funct       
        loss = loss_criterion(outputs, targets)
        train_loss += loss.item()
        loss.backward()
        predicted = outputs.argmax(1)
        correct += predicted.eq(targets).sum().item()

        opt.step()

        #подсчет точности
        total += targets.size(0)

        opt.zero_grad()
        result.append(targets)
        result1.append(outputs)

    del context
    del inputs
    del targets
    gc.collect()
    torch.cuda.empty_cache()

    # запись loss/acc для train в классе statistic, и в Tensorboard

    #         print("epoch---------------")
    num_classes=19
    accuracy_func = torchmetrics.classification.Accuracy(task='multiclass', num_classes=num_classes, average='weighted',
                                                         top_k=3)
    print("train")
    print("epoch: time", datetime.datetime.now() - start_time)
    print("loss: ", train_loss / total)

    # update_result({"loss/train": train_loss / total, "acc/train": correct / total})
    y_train,y_pred = torch.cat(result).reshape(-1).cpu(), torch.cat(result1).cpu()
    update_result({"loss/train": train_loss / total, "acc/train": accuracy_func(y_pred,y_train)})
    


def test(testloader, model, loss_criterion):
    global device
    model.to(device)
    model.eval()

    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, inputs_all in enumerate(testloader):

            inputs, targets, context = None, None, None
            if len(inputs_all) == 2:
                inputs, targets = inputs_all
                inputs = torch.tensor(inputs)
                targets = torch.tensor(targets)
                inputs, targets = inputs.to(device), targets.to(device)
            else:
                inputs, targets, context = inputs_all
                inputs = torch.tensor(inputs)
                targets = torch.tensor(targets)
                context = torch.tensor(context)
                inputs, targets, context = inputs.to(device), targets.to(device), context.to(device)

            outputs = model(inputs, context).reshape(len(inputs), -1)
            #             inputs, targets = inputs.to(device), targets.to(device)
            #             logits = model(inputs)
            #             outputs = torch.nn.functional.log_softmax(logits,dim=1)

            loss = loss_criterion(outputs, targets)

            test_loss += loss.item()
            #             _, predicted = outputs.exp(dim=1).max(1)
            predicted = outputs.argmax(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        del context
        del inputs
        del targets
        gc.collect()
        torch.cuda.empty_cache()

    # запись loss/acc для test в классе statistic, и в Tensorboard
    print('test')
    print("loss: ", test_loss / total, "acc: ", correct / total)
    update_result({"loss/test": test_loss / total, "acc/test": correct / total})

In [36]:
def inference(testloader, model):
    global device
    model.to(device)
    model.eval()
    result = []
    result1 = []

    with torch.no_grad():
        for batch_idx, inputs_all in enumerate(testloader):
            inputs, targets, context = None, None, None
            if len(inputs_all) == 2:
                inputs, targets = inputs_all
                inputs = torch.tensor(inputs)
                targets = torch.tensor(targets)
                inputs, targets = inputs.to(device), targets.to(device)
            else:
                inputs, targets, context = inputs_all
                inputs = torch.tensor(inputs)
                targets = torch.tensor(targets)
                context = torch.tensor(context)
                inputs, targets, context = inputs.to(device), targets.to(device), context.to(device)

            outputs = model(inputs, context).reshape(len(inputs), -1)

            predicted = torch.argmax(outputs, -1)

            result.append(predicted)
            result1.append(outputs)
        del context
        del inputs
        del targets
        gc.collect()
        torch.cuda.empty_cache()

        return torch.cat(result).reshape(-1).cpu(), torch.cat(result1).cpu()


def get_targets(testloader):
    result = []
    for batch_idx, inputs_all in enumerate(testloader):
        inputs, targets, context = None, None, None
        if len(inputs_all) == 2:
            inputs, targets = inputs_all
            targets = torch.tensor(targets)
        else:
            inputs, targets, context = inputs_all
            targets = torch.tensor(targets)
        result.append(targets)
    del context
    del inputs
    del targets
    gc.collect()
    torch.cuda.empty_cache()

    return torch.cat(result).reshape(-1).cpu()

In [37]:
import datetime


def train(epoch, trainloader, testloader, model, loss, opt):
    global current_epoch_number
    global total_epoch
    total_epoch = epoch
    for current_epoch_number in range(epoch):
        print("epoch", current_epoch_number)
        train_epoch(trainloader=trainloader, model=model, opt=opt, loss_criterion=loss)
        test(testloader=testloader, model=model, loss_criterion=loss)

        outputs, y_scores = inference(model=model, testloader=testloader)
        targets = get_targets(testloader=testloader)
        evaluate(targets, outputs, y_scores)

        log()
    outputs,y_scores = inference(model=model, testloader=testloader)
    targets = get_targets(testloader=testloader)
    evaluate_final_stage(outputs, targets, y_scores)

#         if scheduler is None:
#             continue
#         scheduler.step()

In [38]:
# def evaluate(y_true,y_pred):
#     from sklearn.metrics import f1_score
#     from sklearn.metrics import precision_score
#     from sklearn.metrics import recall_score

#     from sklearn.metrics import accuracy_score
#     from sklearn.metrics import confusion_matrix

#     if (y_true<0).all():
#         return
#     # Переписать
#     # Добавить confusion matrix


# #     result = {"acc": accuracy_score(y_pred_class,y_true),
# #               }


#     wandb.log(result,step=current_epoch_number)

In [39]:
# from sklearn.metrics import f1_score
# from sklearn.metrics import precision_score
# from sklearn.metrics import recall_score
# 
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import top_k_accuracy_score
# from sklearn.metrics import confusion_matrix
import torchmetrics


def evaluate(y_true, y_pred, y_scores=None):
    num_classes = 19
    accuracy_func = torchmetrics.classification.Accuracy(task='multiclass', num_classes=num_classes, average='weighted',
                                                         top_k=3)
    precision_func = torchmetrics.classification.Precision(task='multiclass', num_classes=num_classes,
                                                           average='weighted', top_k=3)
    recall_func = torchmetrics.classification.Recall(task='multiclass', num_classes=num_classes, average='weighted',
                                                     top_k=3)
    f1score_func = torchmetrics.classification.F1Score(task='multiclass', num_classes=num_classes, average='weighted',
                                                       top_k=3)

    # if (y_true<0).all():
    #     return
    # 
    # f1_one_vs_all = f1_score(y_pred,y_true,average=None)
    # recall_one_vs_all = recall_score(y_pred,y_true,average=None)
    # recall_one_vs_all = recall_score(y_pred,y_true,average=None)

    result = {"acc_w": accuracy_func(y_scores, y_true),
              "precision": precision_func(y_scores, y_true),
              "recall": recall_func(y_scores, y_true),
              "f1": f1score_func(y_scores, y_true)}

    # result = {"acc": accuracy_score(y_true,y_pred),
    #           "f1_micro":f1_score(y_true,y_pred,average='micro'),
    #           "precision_micro": precision_score(y_true,y_pred,average='micro'),
    #           "recall_micro":recall_score(y_true,y_pred,average='micro'),
    #           "top_k_acc":top_k_accuracy_score(y_true,y_pred,k=3)
    #           }
    # result.update({f"precision_one_vs_all_{i}":val for i,val in enumerate(f1_one_vs_all)})
    # result.update({f"recall_one_vs_all_{i}":val for i,val in enumerate(f1_one_vs_all)})

    #     if y_scores is None:
    #         update_result(result)
    #         return 

    #     result.update({"acc_top_k":top_k_accuracy_score(y_true,y_scores,labels=range(20))})

    update_result(result)


import numpy as np
import pandas as pd
import time


def evaluate_final_stage(y_true, y_pred, y_scores):
    # print(result)
    labels = range(20)
    try:
        table1 = wandb.Table(columns=["tag", "true_tag", "y_scores"], data=y_pred)
        table2 = wandb.Table(columns=["tag", "true_tag", "y_scores"], data= y_true)
        table3 = wandb.Table(columns=["tag", "true_tag", "y_scores"], data=y_scores)
        pd.DataFrame(y_pred.numpy()).to_csv(time.time()+"pred")
        pd.DataFrame(y_true.numpy()).to_csv(time.time()+"true")
        pd.DataFrame(y_scores.numpy()).to_csv(time.time()+"score")

        cm = wandb.plot.confusion_matrix(
            y_true=y_true,
            preds=y_pred,
            class_names=labels)

        wandb.log({"predictions": table1})
        wandb.log({"predictions": table2})
        wandb.log({"predictions": table3})
        wandb.log({"conf_mat": cm})
    except:
        print("error accured")
        

    

## Model

In [40]:
# input_dim, hidden_dim, layer_dim, output_dim

In [41]:
class ToneTagsRNN(torch.nn.Module):
    def __init__(self, embedding, vocab_size=400002, hidden_dim=50, output_size=19, num_layers=2, dropout=0.2,
                 embed_context_size=100, rnn=True):
        super(ToneTagsRNN, self).__init__()
        # vocab_size = 400002
        # embedding_dim = 50
        # hidden_dim_lstm = 30

        # output_size = 19
        self.rnn_output_size = hidden_dim * max_length * 2

        self.embedding = embedding
        self.rnn = torch.nn.RNN(input_size=self.embedding.embedding_dim, hidden_size=hidden_dim, num_layers=num_layers,
                                bidirectional=True, dropout=dropout, batch_first=True)

        #         (64x6400 and 409600x1024)
        #         50*4096*2
        self.fc0 = nn.Linear(embed_context_size, self.embedding.embedding_dim)
        self.fc1 = nn.Linear(hidden_dim * max_length * 2, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, output_size)
        # self.out = nn.Softmax(output_size, dim=1)

    def forward(self, x, context=None):
        activation = torch.nn.ReLU()
        embedded = self.embedding(x)
        if not (context is None):
            embedded = torch.cat(embedded, context)
            embedded = self.fc0(embedded)

        output, hidden = self.rnn(embedded)

        rnn_out = output.reshape(-1, self.rnn_output_size)

        fc1_out = activation(self.fc1(rnn_out))

        fc2_out = activation(self.fc2(fc1_out))
        out = self.fc3(fc2_out)
        #         
        return out

# embedding = nn.Embedding.from_pretrained(embed_tensor, freeze=True)
# vocab_size = len(vocab)
# model = ToneTagsRNN(embedding=embedding)

In [42]:
class ToneTagsLSTM(torch.nn.Module):
    def __init__(self, embedding, vocab_size=400002, hidden_dim=50, output_size=19, num_layers=2, dropout=0.2,
                 embed_context_size=100, rnn=False):
        super(ToneTagsLSTM, self).__init__()
        # vocab_size = 400002
        # embedding_dim = 50
        # hidden_dim_lstm = 30

        # output_size = 19
        self.rnn_output_size = hidden_dim * max_length * 2

        self.embedding = embedding

        self.rnn = torch.nn.LSTM(input_size=self.embedding.embedding_dim, hidden_size=hidden_dim, num_layers=num_layers,
                                 bidirectional=True, dropout=dropout, batch_first=True)

        #         (64x6400 and 409600x1024)
        #         50*4096*2
        self.fc0 = nn.Linear(embed_context_size, self.embedding.embedding_dim)
        self.fc1 = nn.Linear(hidden_dim * max_length * 2, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, output_size)
        # self.out = nn.Softmax(output_size, dim=1)

    def forward(self, x, context=None):
        activation = torch.nn.ReLU()
        embedded = self.embedding(x)
        if not (context is None):
            embedded = torch.cat(embedded, context)
            embedded = self.fc0(embedded)

        output, hidden = self.rnn(embedded)

        rnn_out = output.reshape(-1, self.rnn_output_size)

        fc1_out = activation(self.fc1(rnn_out))

        fc2_out = activation(self.fc2(fc1_out))
        out = self.fc3(fc2_out)
        #         
        return out

# embedding = nn.Embedding.from_pretrained(embed_tensor, freeze=True)
# vocab_size = len(vocab)
# model = ToneTagsRNN(embedding=embedding)

In [43]:
class ToneTags_After_Model_RNN(torch.nn.Module):
    def __init__(self, embedding, vocab_size=400002, hidden_dim=50, output_size=19, num_layers=2, dropout=0.2,
                 embed_context_size=100):
        super(ToneTags_After_Model_RNN, self).__init__()
        # vocab_size = 400002
        # embedding_dim = 50
        # hidden_dim_lstm = 30

        # output_size = 19
        self.rnn_output_size = hidden_dim * max_length * 2

        self.embedding = embedding

        self.seq_layer = torch.nn.RNN(input_size=self.embedding.embedding_dim, hidden_size=hidden_dim,
                                      num_layers=num_layers, bidirectional=True, dropout=dropout, batch_first=True)

        #         (64x6400 and 409600x1024)
        #         50*4096*2
        self.fc0 = nn.Linear(embed_context_size, self.embedding.embedding_dim)
        self.fc1 = nn.Linear(hidden_dim * max_length * 2, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, output_size)

        self.fc4 = nn.Linear(hidden_dim * max_length * 2 + embed_context_size, 1024)
        # self.out = nn.Softmax(output_size, dim=1)

    def forward(self, x, context=None):
        activation = torch.nn.ReLU()
        embedded = self.embedding(x)

        output, hidden = self.seq_layer(embedded)

        rnn_out = output.reshape(-1, self.rnn_output_size)

        # if not (context is None):
        rnn_out = torch.cat(rnn_out, context)
        rnn_out = self.fc4(rnn_out)

        # fc1_out = activation(self.fc1(rnn_out))

        fc2_out = activation(self.fc2(rnn_out))
        out = self.fc3(fc2_out)
        #         
        return out

# embedding = nn.Embedding.from_pretrained(embed_tensor, freeze=True)
# vocab_size = len(vocab)
# model = ToneTagsRNN(embedding=embedding)

In [44]:
class ToneTags_After_Model_LSTM(torch.nn.Module):
    def __init__(self, embedding, vocab_size=400002, hidden_dim=50, output_size=19, num_layers=2, dropout=0.2,
                 embed_context_size=100):
        super(ToneTags_After_Model_LSTM, self).__init__()
        # vocab_size = 400002
        # embedding_dim = 50
        # hidden_dim_lstm = 30

        # output_size = 19
        self.rnn_output_size = hidden_dim * max_length * 2

        self.embedding = embedding

        self.seq_layer = torch.nn.LSTM(input_size=self.embedding.embedding_dim, hidden_size=hidden_dim,
                                       num_layers=num_layers, bidirectional=True, dropout=dropout, batch_first=True)

        #         (64x6400 and 409600x1024)
        #         50*4096*2
        self.fc0 = nn.Linear(embed_context_size, self.embedding.embedding_dim)
        self.fc1 = nn.Linear(hidden_dim * max_length * 2, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, output_size)

        self.fc4 = nn.Linear(hidden_dim * max_length * 2 + embed_context_size, 1024)
        # self.fc4 = nn.Linear(hidden_dim * max_length * 2 + embed_context_size, hidden_dim * max_length * 2)
        # self.out = nn.Softmax(output_size, dim=1)

    def forward(self, x, context=None):
        activation = torch.nn.ReLU()
        embedded = self.embedding(x)

        output, hidden = self.seq_layer(embedded)

        rnn_out = output.reshape(-1, self.rnn_output_size)

        # if not (context is None):
        rnn_out = torch.cat(rnn_out, context)
        rnn_out = self.fc4(rnn_out)

        fc1_out = activation(self.fc1(rnn_out))

        fc2_out = activation(self.fc2(fc1_out))
        out = self.fc3(fc2_out)
        #         
        return out

# embedding = nn.Embedding.from_pretrained(embed_tensor, freeze=True)
# vocab_size = len(vocab)
# model = ToneTagsRNN(embedding=embedding)

In [45]:
# class ToneTagsRNN(torch.nn.Module):
#     def __init__(self,embedding, vocab_size=400002, hidden_dim=50, output_size=19, num_layers=2, dropout=0.4,embed_context_size=100):
#         super(ToneTagsRNN, self).__init__()
#         # vocab_size = 400002
#         # embedding_dim = 50
#         # hidden_dim_lstm = 30

#         # output_size = 19
#         self.rnn_output_size = hidden_dim*max_length*2

#         self.embedding = embedding
#         self.rnn = torch.nn.RNN(input_size=self.embedding.embedding_dim, hidden_size=hidden_dim, num_layers=num_layers, bidirectional=True, dropout=dropout, batch_first=True)
#         self.rnn = torch.nn.RNN(input_size=self.embedding.embedding_dim, hidden_size=hidden_dim, num_layers=num_layers, bidirectional=True, dropout=dropout, batch_first=True)

# #         (64x6400 and 409600x1024)
# #         50*4096*2
#         self.fc0 = nn.Linear(embed_context_size, self.embedding.embedding_dim)
#         self.fc1 = nn.Linear(hidden_dim*max_length*2, 1024)
#         self.fc2 = nn.Linear(1024, 512)
#         self.fc3 = nn.Linear(512, output_size)
#         # self.out = nn.Softmax(output_size, dim=1)

#     def forward(self,x,context=None):
#         activation = torch.nn.ReLU()
#         embedded = self.embedding(x)
#         if not(context is None):
#             embedded = torch.cat(embedded,context)
#             embedded = self.fc0(embedded)

#         output, hidden = self.rnn(embedded)

#         rnn_out = output.reshape(-1, self.rnn_output_size)


#         fc1_out = activation(self.fc1(rnn_out))

#         fc2_out = activation(self.fc2(fc1_out))
#         out = self.fc3(fc2_out)
# #         
#         return torch.nn.Softmax()(out)

# embedding = nn.Embedding.from_pretrained(embed_tensor, freeze=True)
# # vocab_size = len(vocab)
# model = ToneTagsRNN(embedding=embedding)

## Experiments

In [46]:
# configs = {"epoch":20,"optLr":1e-5,"optM":0,"model":"RNN_2","dataset":"dataset_clean","loss": "CrossEntropy"}
# # configs = {"epoch":20,"opt":"Adam","betas":(0.9,0.99),"optLr":5e-4,"optM":0,"model":"ViT","dataset":"noChange","loss": "BCELoss"}

# device = 'cuda' if torch.cuda.is_available() else 'cpu'


# # opt = torch.optim.SGD(params=model.parameters(),lr=configs['optLr'])
# opt = torch.optim.Adam(params=model.parameters(),lr=configs['optLr'])
# loss = torch.nn.CrossEntropyLoss()

# wandb.init(config=configs,
#            project="AML", 
#            name='RNN_2')


# train(epoch=configs['epoch'],trainloader=train_dataloader_clean,testloader=test_dataloader_clean,model=model,loss=loss,opt=opt)


In [47]:
# configs = {"epoch":20,"optLr":1e-3,"optM":0,"model":"RNN_2","dataset":"dataset2","loss": "CrossEntropy"}
# # configs = {"epoch":20,"opt":"Adam","betas":(0.9,0.99),"optLr":5e-4,"optM":0,"model":"ViT","dataset":"noChange","loss": "BCELoss"}

# device = 'cuda' if torch.cuda.is_available() else 'cpu'


# # opt = torch.optim.SGD(params=model.parameters(),lr=configs['optLr'])
# opt = torch.optim.Adam(params=model.parameters(),lr=configs['optLr'])
# loss = torch.nn.CrossEntropyLoss()

# wandb.init(config=configs,
#            project="AML", 
#            name='RNN_2')

# train(epoch=configs['epoch'],trainloader=train_dataloader,testloader=test_dataloader,model=model,loss=loss,opt=opt)


In [48]:
# configs = {"epoch":20,"optLr":1e-3,"optM":0,"model":"RNN_2","dataset":"dataset2","loss": "CrossEntropy"}
# # configs = {"epoch":20,"opt":"Adam","betas":(0.9,0.99),"optLr":5e-4,"optM":0,"model":"ViT","dataset":"noChange","loss": "BCELoss"}

# device = 'cuda' if torch.cuda.is_available() else 'cpu'


# # opt = torch.optim.SGD(params=model.parameters(),lr=configs['optLr'])
# opt = torch.optim.Adam(params=model.parameters(),lr=configs['optLr'])
# loss = torch.nn.CrossEntropyLoss()

# wandb.init(config=configs,
#            project="AML", 
#            name='RNN_2')

# train(epoch=configs['epoch'],trainloader=train_dataloader,testloader=test_dataloader,model=model,loss=loss,opt=opt)


## Tuning

In [49]:
datasets_list = {"clean_corrected_wsd_2": clean_corrected_wsd2_creator,
                 "wsd2": wsd2_creator,
                 }
embedings_list = {
    # "6B":get_glove_6b,
    # "840B":get_glove_42b,
    "twitter": get_glove_twitter
}
models_list = {
    "RNN": ToneTagsRNN,
    "After_Model_RNN": ToneTags_After_Model_RNN,
    # "LSTM":ToneTagsLSTM,
    "After_Model_LSTM": ToneTags_After_Model_LSTM
}


In [50]:
configs1 = {
    "epoch": {'value': 10},
    "optimizer": {'value': 'adam'},

    "betas": {'value': (0.9, 0.99)},

    "optLr": {'values': [1e-5, 1e-4]},

    "dataset": {'values': list(datasets_list.keys())},
    "model": {'values': list(models_list.keys())},
    "embeding": {'values': list(embedings_list.keys())},
    "loss": {'value': "CrossEntropy"}
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'


# # opt = torch.optim.SGD(params=model.parameters(),lr=configs['optLr'])
# 


def run_experiment(config=None):
    with wandb.init(config=config):
        try:
            global vocab_collate_fn
            config = wandb.config
    
            embeding, vocab, e_size = embedings_list[config['embeding']]()
            vocab_collate_fn = vocab
    
            model = models_list[config['model']]
            model = model(embeding, vocab_size=e_size)
    
            opt = torch.optim.Adam(params=model.parameters(), lr=config['optLr'])
    
            loss = torch.nn.CrossEntropyLoss()
    
            train_loader, test_loader = datasets_list[config['dataset']](vocab)
    
            train(epoch=config['epoch'], trainloader=train_loader, testloader=test_loader, model=model, loss=loss, opt=opt)
    
            del train_loader
            del test_loader
            del embeding
            del vocab
    
            vocab_collate_fn = None
    
            gc.collect()
            torch.cuda.empty_cache()
        except:
            print("Error")


sweep_config = {
    'method': 'grid'
}
sweep_config['metric'] = {'name': 'loss/train',
                          'goal': 'minimize'
                          }
sweep_config['parameters'] = configs1
sweep_id1 = wandb.sweep(sweep_config, project="AML")
# sweep_config['parameters'] = configs2
# sweep_id2 = wandb.sweep(sweep_config, project="kontur")

wandb.agent(sweep_id1, run_experiment)
# wandb.agent(sweep_id2, run_experiment)

Create sweep with ID: wogrht6r
Sweep URL: https://wandb.ai/my_own_opt/AML/sweeps/wogrht6r


wandb: Agent Starting Run: 3w5lkobh with config:
wandb: 	betas: [0.9, 0.99]
wandb: 	dataset: clean_corrected_wsd_2
wandb: 	embeding: twitter
wandb: 	epoch: 10
wandb: 	loss: CrossEntropy
wandb: 	model: RNN
wandb: 	optLr: 1e-05
wandb: 	optimizer: adam


epoch 0
train
epoch: time 0:02:54.969089
loss:  0.08323159860205774
test
loss:  0.08305125929555802 acc:  0.16892688679245282
epoch 1
train
epoch: time 0:02:46.756752
loss:  0.08305116687399904
test
loss:  0.08299419688728621 acc:  0.16898584905660377
epoch 2
train
epoch: time 0:02:47.953356
loss:  0.08298604416007767
test
loss:  0.08297510012140814 acc:  0.16774764150943397
epoch 3
train
epoch: time 0:02:48.412479
loss:  0.08292873188663753
test
loss:  0.08298668462150502 acc:  0.16810141509433962
epoch 4
train
epoch: time 0:02:49.877820
loss:  0.08283913292401364
test
loss:  0.0829818123354102 acc:  0.16721698113207548
epoch 5
train
epoch: time 0:02:50.307178
loss:  0.08263179245025438
test
loss:  0.08304829012672856 acc:  0.16780660377358492
epoch 6
train
epoch: time 0:02:50.670029
loss:  0.08232693103577694
test
loss:  0.08308612028101706 acc:  0.16751179245283018
epoch 7
train
epoch: time 0:02:51.649722
loss:  0.08191910725450156
test
loss:  0.0831830841612141 acc:  0.166686320754

acc/test,██▆▆▅▆▅▄▆▁
acc/train,▁▂▂▂▂▃▄▅▇█
acc_w,██████▁▇▆▆
f1,▁▁▂▁▂▂▇▄▄█
loss/test,▂▁▁▁▁▂▂▃▄█
loss/train,█▇▇▇▇▆▅▄▃▁
precision,▃▄▂▁▂█▂▄▅▅
recall,██████▁▇▆▆
acc/test,0.16498
acc/train,0.43289
acc_w,0.40973


wandb: Agent Starting Run: 9u4zx6l2 with config:
wandb: 	betas: [0.9, 0.99]
wandb: 	dataset: clean_corrected_wsd_2
wandb: 	embeding: twitter
wandb: 	epoch: 10
wandb: 	loss: CrossEntropy
wandb: 	model: RNN
wandb: 	optLr: 0.0001
wandb: 	optimizer: adam


...

wandb: Agent Starting Run: g1hd30ip with config:
wandb: 	betas: [0.9, 0.99]
wandb: 	dataset: wsd2
wandb: 	embeding: twitter
wandb: 	epoch: 10
wandb: 	loss: CrossEntropy
wandb: 	model: After_Model_LSTM
wandb: 	optLr: 1e-05
wandb: 	optimizer: adam


epoch 0
Error


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 4skzpax9 with config:
wandb: 	betas: [0.9, 0.99]
wandb: 	dataset: wsd2
wandb: 	embeding: twitter
wandb: 	epoch: 10
wandb: 	loss: CrossEntropy
wandb: 	model: After_Model_LSTM
wandb: 	optLr: 0.0001
wandb: 	optimizer: adam


epoch 0
Error


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [51]:
print('Hello world')

Hello world
